In [53]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import csv

# Google developer API key
from config import gkey

In [59]:
# 6. Bonus: Find the names and addresses of the top five restaurants in your home city.
#    Hint: Read about "Text Search Results"
# (https://developers.google.com/places/web-service/search#TextSearchRequests)
my_phrase = "Fitness Miami"
target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

params = {
    "query": my_phrase,
    "key": gkey
}

print("Fitness centers")

response = requests.get(target_url, params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)
fitness_centers = response.json()
# print(json.dumps(happy_places, indent=4, sort_keys=True))

for place in fitness_centers["results"]:
    print(place["name"])
    print(place["formatted_address"])

Fitness centers
Core Fitness Miami
162 NE 25th St, Miami, FL 33137, USA
LA Fitness
3301 SW 22nd St, Miami, FL 33145, USA
B-Fit Biscayne
5080 Biscayne Blvd, Suite 1-B, Miami, FL 33137, United States
LA Fitness
900 S Miami Ave #236, Miami, FL 33130, USA
Orangetheory Fitness
244 Biscayne Blvd, Miami, FL 33132, USA
Punch Elite Fitness
1901 NW Miami Ct, Miami, FL 33136, USA
LegacyFit
77 NE 24th St, Miami, FL 33137, USA
Bodytek Fitness Wynwood - Miami
545 NW 28th St, Miami, FL 33127, USA
Orangetheory Fitness
9017 Biscayne Blvd, Miami Shores, FL 33138, USA
24 Hour Fitness
2982 Grand Ave, Miami, FL 33133, USA
Orangetheory Fitness
89 NE 38th St, Miami, FL 33137, USA
Equinox South Beach
520 Collins Ave, Miami Beach, FL 33139, USA
ERA Fit
1625 N Miami Ave, Miami, FL 33136, USA
Ironflower Fitness
7205 NE 4th Ave, Miami, FL 33138, USA
Primal Fit 360 Personal Training
9531 NE 2nd Ave, Miami Shores, FL 33138, USA
DBC Fitness
3841 NE 2nd Ave #103, Miami, FL 33137, USA
Society Fitness
Riverfront, 92 SW

In [55]:
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
fitness_centers.df=pd.DataFrame([fitness_centers])

AttributeError: 'dict' object has no attribute 'df'